# Preprocessing

## Config

In [2]:
PHASE_CLASSES = 4
N_FRAMES = 10
BATCH_SIZE = 8

## Mount Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/Individual Models FXAI/

/content/drive/MyDrive/Individual Models FXAI


## Env setup

In [5]:
# The way this tutorial uses the `TimeDistributed` layer requires TF>=2.10
#!pip install -Uq "tensorflow>=2.10.0"
!pip install --quiet --upgrade tensorflow-federated

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.4/114.4 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.5/558.5 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 38.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.7/33.7 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [6]:
!pip install remotezip tqdm opencv-python
!pip install -q git+https://github.com/tensorflow/docs

  Preparing metadata (setup.py) ... done
  Created wheel for remotezip: filename=remotezip-0.12.1-py3-none-any.whl size=7933 sha256=643a3f16d712fbb3617d38af5093feb6853217c1b7f74a0a1ed5f0520108ac0a
  Stored in directory: /root/.cache/pip/wheels/fc/76/04/beed1a6df4eb7430ee13c3900746edd517e5e597298d1f73f3
Successfully built remotezip
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 2.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dp-accounting 0.4.2 requires attrs~=21.4, but you have attrs 23.1.0 which is incompatible.
google-vizier 0.1.4 requires attrs==21.4.0, but you have attrs 23.1.0 which is incompatible.
tensorflow-federated 0.61.0 requires attrs~=21.4, but you have attrs 23.1.0 which is incompatible.
tensorflow-privacy 0.8.10 requires attrs~=21.4, but you have attrs 23.1.0 which is incom

In [7]:
import tqdm
import random
import pathlib
import itertools
import collections

import os
import cv2
import numpy as np
import remotezip as rz

import tensorflow as tf

# Some modules to display an animation using imageio.
import imageio
from IPython import display
from urllib import request
from tensorflow_docs.vis import embed

# extras
from sklearn.model_selection import train_test_split
import keras
from keras.layers.convolutional import ( Conv2D, MaxPooling2D, AveragePooling2D)
from keras.layers import (    Input,    Activation,    Dense,    Flatten)
from keras.layers import add
from keras.layers import LayerNormalization

from keras.regularizers import l2
from keras import backend as K
from keras.models import Model

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.utils import np_utils

## Load data

In [8]:
NPZ_Name = 'Data/Videos_Database_20_Robot_WebCam_50_overall_database.npz'
Database_Used = np.load(NPZ_Name)
Sessions = Database_Used['Session']

In [9]:
# create a list of the unique sessions to become the client_ids
client_ids = np.unique(Sessions)

### Get into format of classes and video files

In [10]:
# Using code from tff to sort data
def create_fg_dataset(client_ids, Database_used_col, categorical, categories):
  dataset = [[] for _ in range(categories)]
  dataset_train = [[] for _ in range(categories)]
  dataset_test = [[] for _ in range(categories)]
  for session in client_ids:
    # find the indices of the current session in the Sessions column of Database_Used
    session_indices = np.where(Sessions == session)[0]

    # get the X_train data for the current session
    session_X = Database_Used['X_train'][session_indices]
    # grab the training data for the necessary hierarchy
    session_Y = Database_Used[Database_used_col][session_indices]
    # if using categorical data, reshape the data for the model into one-hot encoded
    # if categorical==True:
    #   session_Y = np_utils.to_categorical(session_Y, categories)

    '''
    Here is where we need to further split on classes
    '''
    # The session dataset will be a list of lists
    # The first index will represent the class
    # The second index will represent the frames in the session belonging to that class

    # get the unique classes in the session
    session_classes = np.unique(session_Y)

    # for each class in the session
    for session_class in session_classes:
        # get the indices of the current class
        class_indices = np.where(session_Y == session_class)[0]
        # get the X_train data for the current class
        class_X = session_X[class_indices]
        # append the class data to the dataset, using the class as the index
        dataset[session_class].append(class_X)

  # create a train/test split of the dataset for each class
  for i in range(categories):
      dataset_train[i], dataset_test[i] = train_test_split(dataset[i], test_size=0.2)

  return dataset_train, dataset_test

'''
The dataset is a list of lists of lists
index 0: class
index 1: video selection
index 2: frame selection
'''

'\nThe dataset is a list of lists of lists\nindex 0: class\nindex 1: video selection\nindex 2: frame selection\n'

In [11]:
X_train, X_test = create_fg_dataset(client_ids, 'Y_train_Context', True, PHASE_CLASSES)

### Shorten the video files appropriately

In [12]:
def frames_from_video_file(video_frames, n_frames, frame_step = 15):

  # Read each video frame by frame
  result = []

  video_length = len(video_frames)

  # Calculate the minimum required length for the video to get the desired
  # number of frames considering the frame step
  need_length = 1 + (n_frames - 1) * frame_step

  # Either start at the beginning or at a random point between the beginning
  # and the latest possible point to ensure the desired number of frames
  # can be gathered
  if need_length > video_length:
    start = 0
  else:
    max_start = video_length - need_length
    start = random.randint(0, max_start + 1)

  # Begin collecting the frames
  frame = video_frames[start]
  result.append(frame)

  current = start

  for _ in range(n_frames - 1):
    current += frame_step
    # If we have run out of space in the array, just fill it with black
    if current >= video_length:
      result.append(np.zeros_like(result[0]))
    else:
      frame = video_frames[current]
      result.append(frame)

  return result

In [13]:
# Need to get all the frames sorted pre-FrameGenerator
for i in range(len(X_train)):
  for j in range(len(X_train[i])):
    X_train[i][j] = frames_from_video_file(X_train[i][j], N_FRAMES)
    X_train[i][j] = np.array(X_train[i][j])

for i in range(len(X_test)):
  for j in range(len(X_test[i])):
    X_test[i][j] = frames_from_video_file(X_test[i][j], N_FRAMES)
    X_test[i][j] = np.array(X_test[i][j])

In [14]:
X_train[0][0].shape

(10, 50, 50, 3)

In [15]:
class FrameGenerator:
  def __init__(self, dataset, training = False):
    """ Returns a set of frames with their associated label.

      Args:
        path: Video file paths.
        n_frames: Number of frames.
        training: Boolean to determine if training dataset is being created.
    """
    self.dataset = dataset
    self.training = training
    self.class_names = [i for i in range(len(dataset))]

  def get_videos_and_labels(self):
    videos = []
    labels = []
    for i in range(len(self.dataset)):
      for j in range(len(self.dataset[i])):
        videos.append(self.dataset[i][j])
        labels.append(i)
    return videos, labels

  def __call__(self):
    # No longer using the training boolean to shuffle data, not an issue just
    # something to be aware of

    videos, labels = self.get_videos_and_labels()

    pairs = list(zip(videos, labels))

    if self.training:
      random.shuffle(pairs)

    for video_frames, label in pairs:
      yield video_frames, label

In [16]:
# Test out the FrameGenerator
fg = FrameGenerator(X_train)

frames, label = next(fg())

print(f"Shape: {frames.shape}")
print(f"Label: {label}")

Shape: (10, 50, 50, 3)
Label: 0


In [17]:
# Create the training set
output_signature = (tf.TensorSpec(shape = (None, None, None, 3), dtype = tf.float32),
                    tf.TensorSpec(shape = (), dtype = tf.int16))
train_ds = tf.data.Dataset.from_generator(FrameGenerator(X_train, training=True),
                                          output_signature = output_signature)

In [18]:
# Making sure they are shuffled
for frames, labels in train_ds.take(10):
  print(labels)

tf.Tensor(2, shape=(), dtype=int16)
tf.Tensor(3, shape=(), dtype=int16)
tf.Tensor(3, shape=(), dtype=int16)
tf.Tensor(0, shape=(), dtype=int16)
tf.Tensor(3, shape=(), dtype=int16)
tf.Tensor(1, shape=(), dtype=int16)
tf.Tensor(0, shape=(), dtype=int16)
tf.Tensor(3, shape=(), dtype=int16)
tf.Tensor(2, shape=(), dtype=int16)
tf.Tensor(1, shape=(), dtype=int16)


In [19]:
# Create test set
test_ds = tf.data.Dataset.from_generator(FrameGenerator(X_test, training=True),
                                          output_signature = output_signature)

In [20]:
# Print the shapes of the data
train_frames, train_labels = next(iter(train_ds))
print(f'Shape of training set of frames: {train_frames.shape}')
print(f'Shape of training labels: {train_labels.shape}')

test_frames, test_labels = next(iter(test_ds))
print(f'Shape of validation set of frames: {test_frames.shape}')
print(f'Shape of validation labels: {test_labels.shape}')

Shape of training set of frames: (10, 50, 50, 3)
Shape of training labels: ()
Shape of validation set of frames: (10, 50, 50, 3)
Shape of validation labels: ()


In [21]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)

In [22]:
train_ds = train_ds.batch(BATCH_SIZE)
test_ds = test_ds.batch(BATCH_SIZE)

train_frames, train_labels = next(iter(train_ds))
print(f'Shape of training set of frames: {train_frames.shape}')
print(f'Shape of training labels: {train_labels.shape}')

test_frames, test_labels = next(iter(test_ds))
print(f'Shape of validation set of frames: {test_frames.shape}')
print(f'Shape of validation labels: {test_labels.shape}')

Shape of training set of frames: (8, 10, 50, 50, 3)
Shape of training labels: (8,)
Shape of validation set of frames: (8, 10, 50, 50, 3)
Shape of validation labels: (8,)
